# Comparison of Two Time Series of Maps 1.0
*A concise description of purpose and scope*

## Table of Contents  
1. [Environment Setup](#environment-setup)
2. [Toy Data Input Format](#data-preparation)
3. [Presence Agreement Components (Eqs 1–12)](#presence-agreement)
4. [Change Components: Gains & Losses (Eqs 13–28)](#change-components)
5. [Full-Extent Change Metrics (Eqs 29–40, 41–52)](#full-extent)
6. [Visualization of Results](#visualization)
7. [Exporting Results](#export)


## 1. Environment Setup <a id="environment-setup"></a>

### 1.1 Install Dependencies  

### 1.2 Import Libraries  

### 1.3 Define Constants & Settings

## 2. Toy Data Input Format <a id="data-preparation"></a>

### 2.1 Generate or Load Toy Time Series Array  

## 3. Presence Agreement Components (Eqs 1–12) <a id="presence-agreement"></a>

### 3.1 Define Presence Variables:

### 3.2 Implement Hit/Miss/False-Alarm/Spatial-Diff/Temporal-Diff Functions  

### 3.3 Compute Component Arrays per Time & Pixel  

### 3.4 Aggregate Presence Components 

## 4. Change Components: Gains & Losses (Eqs 13–28) <a id="change-components"></a>

### 4.1 Define Gain & Loss Variables:  

### 4.2 Implement Gain-Component Functions  

### 4.3 Implement Loss-Component Functions 

### 4.4 Aggregate Gain & Loss Components  

## 5. Full-Extent Change Metrics (Eqs 29–40, 41–52) <a id="full-extent"></a>

### 5.1 Compute Total Gains & Losses over \(t=0\to T\)  

### 5.2 Quantity Metrics (Interval & Full-Extent Sums)  

### 5.3 Pixel-Wise Summary Metrics  

## 6. Visualization of Results <a id="visualization"></a>

### 6.1 Stacked Bar Chart: Presence Agreement  

### 6.2 Stacked Bar Chart: Gain & Loss Components  

### 6.3 Composition of Full-Extent Change  

## 7. Exporting Results <a id="export"></a>

### 7.1 Save Metrics DataFrame to CSV/Excel  

### 7.2 Save Figures (PNG)  